In [1]:
# BERT 모델 사용

import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

2023-02-24 00:06:59.315659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 00:06:59.467601: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 00:06:59.467621: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-24 00:07:00.469239: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [ ]:
# GPU가 맞게 설정됐는지 확인하는 코드
'''
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))
'''

In [2]:
# 분류 모델 파인튜닝을 위해 Pretrained BERT에 추가 데이터셋 학습 필요

import csv
import pandas as pd

# 데이터프레임으로 변환
chatbot_data = pd.read_csv('ChatbotData.csv',encoding="utf-8")
print(chatbot_data.shape)
print(chatbot_data.sample(n=5))

(11823, 3)
                         Q                     A  label
10186   썸 타는 노래 듣는 중인데 좋아.         썸 탈 때 듣기 딱이죠.      2
8544   헤어지고서 10개월동안 4번 봤네.               많이 봤네요.      1
3345            오늘은 또 뭐 입냐         아침마다 하는 고민이죠.      0
8096           제 친구가 그러더라구  친구보다 자신의 마음에 귀기울이세요.      1
3617             은퇴후가 걱정이야           인생이 꽤 기니까요.      0


In [3]:
# 현재 3개의 클래스로 라벨링 되어 있음
# 이진분류가 목표이기 때문에 '1'과 '2'로 라벨링 되어 있는 데이터를 모두 '1'로 변경
chatbot_data.loc[(chatbot_data['label'] == 2), 'label'] = 1  #라벨 1과 2는 모두 1로 통일

# 현재 데이터셋은 0~5290 까지의 행이 모두 클래스가 '0'이고 5291행부터 마지막 행까지 모두 클래스가 '1' 임
# 데이터셋을 클래스가 '0'인 데이터와 '1'인 데이터로 랜덤으로 골고루 섞는 코드
chatbot_data_shuffled = chatbot_data.sample(frac=1).reset_index(drop=True)
print(chatbot_data_shuffled.shape)

#train data & test data 로드 
train = chatbot_data_shuffled[:9000]
test = chatbot_data_shuffled[9000:]
print(train.sample(n=5))

(11823, 3)
                        Q                      A  label
6212                불면증인가  생각이 너무 많아서 그런 것 같습니다.      0
648                    텅빔            다시 채워질 거예요.      1
2443         마음에 하나도 안 들어              그럴 때가 있죠.      0
7580         방금 헤어지고 왔습니다              맘고생 많았어요.      1
7962  이별 일주일 후 ! 우리의 이야기.        각자의 삶을 살고 있겠지요.      1


In [4]:
# 현재 데이터셋을 BERT 모델의 입력 데이터 형식으로 변경하기 위한 전처리

# BERT 분류 모델의 경우 : [CLS] = 문장의 처음, [SEP] = 문장의 끝
# 각 문장의 앞마다 [CLS]를 붙이는 작업 필요, 문장의 종료는 [SEP]를 붙임

# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in train.Q]
print(len(sentences))
print(sentences[:5])

# '0'과 '1'의 라벨이 들어있는 컬럼을 'labels'이라는 array에 따로 저장
labels = train['label'].values
print(labels[:5])

9000
['[CLS] 여친 생겼는데 호칭 어떻게 함? [SEP]', '[CLS] 정시 퇴근하고 집에 가는 중 [SEP]', '[CLS] 짝남은 나한테 관심이 없는 것 같애. [SEP]', '[CLS] 여자친구가 너무 무뚝뚝해 [SEP]', '[CLS] 어렸을때 공부할껄 [SEP]']
[1 0 1 1 0]


In [5]:
import pandas as pd
from transformers import BertTokenizer

# BERT Tokenizer 테스트
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('지바이크')
print(result)

# BERT Tokenizer (WordPiece)를 이용하여 전체 데이터에 토크나이징을 수행
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]
print(len(tokenized_texts))

print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

# 문장의 최대 시퀀스를 설정하고 정수 인코딩과 제로 패딩 수행
#MAX_LEN = 512 #BERT에서 지원하는 문장의 최대 길이 = 512
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids.shape)
print(input_ids[0])

['지', '##바', '##이', '##크']
9000
[CLS] 여친 생겼는데 호칭 어떻게 함? [SEP]
['[CLS]', '여', '##친', '생', '##겼', '##는데', '호', '##칭', '어', '##떻', '##게', '함', '?', '[SEP]']
(9000, 128)
[   101   9565  55358   9420 118637  41850   9985  52094   9546 118837
  14153   9956    136    102      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0    

In [6]:
# Attention Mask (어텐션 마스크) 필요
# 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록
# 단어와 패딩 토큰을 구분할 수 있도록 패딩된 데이터가 있을 때 위치에 따라,
# 패딩된 값은 '0', 패딩되지 않은 단어는 '1'을 갖는 시리얼 데이터를 생성
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(len(attention_masks))
print(attention_masks[0])

# 결과 예) 패딩된 데이터 = [40311, 9435, 102, 0, 0]
# >>어텐션 마스크 = [1.0, 1.0, 1.0, 0.0, 0.0]

9000
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [7]:
# 데이터를 모두 파이토치 텐서로 변환
# Train Set 전처리 : 어텐선 마스크를 포함 데이터셋을 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)     
                                                       
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [8]:
# Test Set 전처리
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.Q]

# 라벨 데이터
labels = test['label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
#MAX_LEN = 512 #BERT에서 지원하는 문장의 최대 길이 = 512
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [9]:
# BERT Model Loading
'''
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')
'''
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
#model.cuda()

# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [10]:
# BERT Model Training
# Note : Tesla T4 generally has a higher number of cores and higher memory bandwidth than an Intel CPU
# On a machine with a modern CPU, 16GB RAM, and no GPU, fine-tuning a small BERT model (e.g., BERT-base)
# on a moderate-sized dataset (e.g., 10,000 examples) could take several days to a week (7 days)!

# 주의사항 : GPU 없는 서버(AI Live Server)에서 GPU 코드 실행시 서버 다운 될 수 있음!
# GPU 없이 CPU 8개로 학습할 경우 : 1 에폭 학습에 약 24분 걸림 (데이터셋 사이즈 9000 문장)
# Tesla T4 GPU 1개의 경우 : 1 에폭 학습에 약 3분 걸림 (데이터셋 사이즈 9000 문장)

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

#랜덤시드 고정
seed_val = 1234
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        #batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        #batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0] # The raw outputs of a layer (the final layer)

        # CPU로 데이터 이동
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()
        label_ids = b_labels.numpy()
        logits = logits.numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("======================")
print("= Training complete! =")
print("======================")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.44
  Training epcoh took: 0:23:23

Running Validation...
  Accuracy: 0.84
  Validation took: 0:00:46

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.31
  Training epcoh took: 0:23:39

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:46

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:23:32

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:46

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.17
  Training epcoh took: 0:23:24

Running Validation...
  Accuracy: 0.87
  Validation took: 0:00:45
= Training complete! =


In [11]:
# 테스트셋 평가
# GPU 없이 CPU 8개로 학습할 경우 : 결과 예측에 약 2분 걸림 (데이터셋 사이즈 약 2000 문장)
# Tesla T4 GPU 1개의 경우 : 결과 예측에 약 0.2초 걸림 (데이터셋 사이즈 약 2000 문장)

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    #batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0] # The raw outputs of a layer (the final layer)

    # CPU로 데이터 이동
    #logits = logits.detach().cpu().numpy()
    #label_ids = b_labels.to('cpu').numpy()
    label_ids = b_labels.numpy()
    logits = logits.numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.88
Test took: 0:02:23


In [19]:
# 새로운 문장 테스트

# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    #MAX_LEN = 512 #BERT에서 지원하는 문장의 최대 길이 = 512
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    #b_input_ids = inputs.to(device)
    #b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        '''
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        '''
        outputs = model(inputs, 
                        token_type_ids=None, 
                        attention_mask=masks)

    # 로스 구함
    logits = outputs[0] # The raw outputs of a layer (the final layer)
    logits = logits.numpy()

    # CPU로 데이터 이동
    #logits = logits.detach().cpu().numpy()

    return logits

In [21]:
# 새로운 문장 테스트 (샘플)

logits = test_sentences(['지쿠터 타고갈까'])
print(logits)

if np.argmax(logits) == 1 :
    print("1번 타입 대화")
elif np.argmax(logits) == 0 :
    print("0번 타입 대화")

[[-1.1652505  1.2381825]]
1번 타입 대화
